# Solving DTLZ1 using MOEA/D with two scalarization functions

In this example we are going to solve the DTLZ1 benchmark problem using MOEA/D. To this aim, we need to import the following packages:


In [34]:
# Package for plotting the results
import plotly.graph_objects as go

# Package to create the Problem object
from desdeo_problem.testproblems.TestProblems import test_problem_builder

# Packages for using MOEA/D and the available scalarization functions
from desdeo_emo.EAs.MOEAD import MOEA_D
from desdeo_tools.scalarization.MOEADSF import (
    MOEADSFBase,
    Tchebycheff,
    PBI,
    WeightedSum,
)

Then, we will use the implementation of DTLZ1 available in desdeo-problem. We will use 7 decision variables and 3 objectives.

In [35]:
problem = test_problem_builder(name="DTLZ1", n_of_variables=7, n_of_objectives=3)

Now we need to pass some parameters to the MOEA/D: the problem to solve, and the number of generations. The rest of the parameters will take the default values, for more information about the available options, please refer to the API documentation. 

In [36]:
evolver = MOEA_D(problem, n_iterations=1, n_gen_per_iter=300)

Finally, we create a loop to run the algorithm until the stop criteria is met.

In [37]:
while evolver.continue_evolution():
    evolver.iterate()

To visualize the results, we will plot the values of the objective function of the final population. 

In [38]:
objectives = evolver.population.objectives
fig2 = go.Figure(data=go.Scatter3d(x=objectives[:,0],
                                   y=objectives[:,1],
                                   z=objectives[:,2],
                                   mode="markers",
                                   marker_size=1))
fig2

The previous example used the default scalarization function of the MOEA/D implementation, the PBI function. In the following example, we will use the Tchebycheff function. First we create a Tchebycheff object to use the implementation of this scalarizaion function available in desdeo-tools. Then, we use this object in the parameter "scalarization_function" of MOEA/D and run the algorithm until the stop criteria is met. 

In [39]:
tch =  Tchebycheff()
evolver = MOEA_D(problem, scalarization_function = tch, n_iterations=1, n_gen_per_iter=300)

while evolver.continue_evolution():
    evolver.iterate()

To visualize the new results, we create another plot with the objective values. 

In [40]:
objectives = evolver.population.objectives
fig3 = go.Figure(data=go.Scatter3d(x=objectives[:,0],
                                   y=objectives[:,1],
                                   z=objectives[:,2],
                                   mode="markers",
                                   marker_size=1))
fig3

The solutions in this example have a diffeent distribution that the ones in the first figure.
